In [1]:
'''libray imports'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import joblib

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import (SVC, SVR)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, precision_recall_curve
)
from sklearn.calibration import calibration_curve

from sklearn.model_selection import train_test_split


In [2]:
# This cell loads the preprocessed feature matrices and target labels
# that were previously saved to disk to avoid repeating preprocessing.

X_train = pd.read_parquet("data/processed/X_train.parquet")
X_val   = pd.read_parquet("data/processed/X_val.parquet")

y_train = pd.read_parquet("data/processed/y_train.parquet")["reordered"]
y_val   = pd.read_parquet("data/processed/y_val.parquet")["reordered"]

print("X_train:", X_train.shape, "X_val:", X_val.shape)
print("y_train:", y_train.shape, "y_val:", y_val.shape)
print("Class balance (train):")
print(y_train.value_counts(normalize=True))


X_train: (25947591, 8) X_val: (6486898, 8)
y_train: (25947591,) y_val: (6486898,)
Class balance (train):
reordered
1    0.589697
0    0.410303
Name: proportion, dtype: float64


In [3]:
'''CLASSIFICATION: TASK(A)'''
# This cell performs basic sanity checks on the data
X_train = X_train.copy()
X_val   = X_val.copy()

if "recent_reorder_rate" in X_train.columns:
    X_train.loc[:, "recent_reorder_rate"] = X_train["recent_reorder_rate"].fillna(0.0)
if "recent_reorder_rate" in X_val.columns:
    X_val.loc[:, "recent_reorder_rate"] = X_val["recent_reorder_rate"].fillna(0.0)

print("NaN in X_train:", X_train.isna().any().any())
print("NaN in X_val:  ", X_val.isna().any().any())


NaN in X_train: False
NaN in X_val:   False


In [4]:
def get_pred_and_proba(model, X):
    """
    Returns (y_pred, y_score) where y_score is probability (preferred) or a [0,1] score.
    """
    y_pred = model.predict(X)

    if hasattr(model, "predict_proba"):
        y_score = model.predict_proba(X)[:, 1]
        return y_pred, y_score

    if hasattr(model, "decision_function"):
        scores = model.decision_function(X)
        scores = (scores - scores.min()) / (scores.max() - scores.min() + 1e-12)
        return y_pred, scores

    raise ValueError("Model must support predict_proba or decision_function.")


In [ ]:
# It computes accuracy, precision, recall, F1-score, ROC-AUC,
# Precision-Recall AUC, confusion matrices, and calibration curves.
#please keep in mind that if you want to see the plots make the (show_plots=True)
def evaluate_model(model, X, y, name, show_plots=False):
    y_pred, y_score = get_pred_and_proba(model, X)

    acc  = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred, zero_division=0)
    rec  = recall_score(y, y_pred, zero_division=0)
    f1   = f1_score(y, y_pred, zero_division=0)

    roc_auc = roc_auc_score(y, y_score)
    ap      = average_precision_score(y, y_score)

    # Confusion matrices
    cm_raw = confusion_matrix(y, y_pred)
    cm_norm = confusion_matrix(y, y_pred, normalize="true")

    if show_plots:
        # Confusion Matrix raw
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_raw)
        disp.plot(values_format="d")
        plt.title(f"{name} — Confusion Matrix (raw)")
        plt.show()

        # Confusion Matrix normalized
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_norm)
        disp.plot(values_format=".2f")
        plt.title(f"{name} — Confusion Matrix (normalized)")
        plt.show()

        # ROC
        fpr, tpr, _ = roc_curve(y, y_score)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
        plt.plot([0,1],[0,1], linestyle="--")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"{name} — ROC Curve")
        plt.legend()
        plt.show()

        # PR
        p, r, _ = precision_recall_curve(y, y_score)
        plt.figure()
        plt.plot(r, p, label=f"AP={ap:.3f}")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title(f"{name} — Precision-Recall Curve")
        plt.legend()
        plt.show()

        # Calibration (Reliability Diagram)
        prob_true, prob_pred = calibration_curve(y, y_score, n_bins=10, strategy="uniform")
        plt.figure()
        plt.plot(prob_pred, prob_true, marker="o")
        plt.plot([0,1],[0,1], linestyle="--")
        plt.xlabel("Mean predicted probability")
        plt.ylabel("Fraction of positives")
        plt.title(f"{name} — Calibration Curve")
        plt.show()

    return {
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "ROC_AUC": roc_auc,
        "AP": ap,
        "CM_raw": cm_raw,
        "CM_norm": cm_norm,
        "N_eval": len(X)
    }


In [6]:
def stratified_subsample(X, y, n, random_state=42):
    n = min(n, len(X))
    X_s, _, y_s, _ = train_test_split(
        X, y, train_size=n, stratify=y, random_state=random_state
    )
    return X_s, y_s


In [16]:
# This cell defines and trains a Logistic Regression model
logreg_l2 = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0.0)),
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        solver="saga",
        class_weight="balanced",
        max_iter=500,
        random_state=42
    ))
])
logreg_l2.fit(X_train, y_train)

C:\Users\zaidg\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('imputer', ...), ('scaler', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'constant'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",0.0
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. 

In [17]:
# This cell defines the KNN classifier.
# Due to its high computational cost, it will later be evaluated
# on a validation subsample only.
knn = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0.0)),
    ("scaler", StandardScaler()),
    ("clf", KNeighborsClassifier(n_neighbors=25))
])

X_knn_tr, y_knn_tr = stratified_subsample(X_train, y_train, n=300_000)
knn.fit(X_knn_tr, y_knn_tr)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('imputer', ...), ('scaler', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'constant'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",0.0
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. 

In [18]:
#This cell trains a Decision Tree classifier.
dt = DecisionTreeClassifier(
    max_depth=10,
    min_samples_leaf=50,
    class_weight="balanced",
    random_state=42
)
dt.fit(X_train, y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",50
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current 

In [19]:
from sklearn.model_selection import train_test_split

SAMPLE_N = 1_000_000

X_rf, _, y_rf, _ = train_test_split(
    X_train, y_train,
    train_size=min(SAMPLE_N, len(X_train)),
    stratify=y_train,
    random_state=42
)

print(X_rf.shape, y_rf.value_counts(normalize=True))


(1000000, 8) reordered
1    0.589697
0    0.410303
Name: proportion, dtype: float64


In [20]:
# This cell trains a Random Forest classifier.
X_rf_tr, y_rf_tr = stratified_subsample(X_train, y_train, n=200_000)

rf = RandomForestClassifier(
    n_estimators=90,
    max_depth=10,
    min_samples_leaf=75,
    max_features="sqrt",
    class_weight="balanced",
    bootstrap=False,
    n_jobs=1,
    random_state=42
)
rf.fit(X_rf_tr, y_rf_tr)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",90
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",75
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",False
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric(

In [22]:
# This cell checks for overfitting by comparing train and validation performance
# We use a subsample of the training data for efficiency

X_tr_small, y_tr_small = stratified_subsample(X_train, y_train, n=300_000)

train_metrics = evaluate_model(
    rf, X_tr_small, y_tr_small, "RF Train Sample", show_plots=False
)

val_metrics = evaluate_model(
    rf, X_val, y_val, "RF Validation", show_plots=False
)

pd.DataFrame([train_metrics, val_metrics])[
    ["Model", "Accuracy", "Precision", "Recall", "F1", "ROC_AUC", "AP"]
]


,Model,Accuracy,Precision,Recall,F1,ROC_AUC,AP
0,RF Train Sample,0.92942,0.930981,0.950800,0.940786,0.984527,0.988922
1,RF Validation,0.92865,0.930667,0.949761,0.940117,0.984018,0.988526


In [23]:
# This cell defines linear and RBF SVM classifiers.
# Training and evaluation are performed on subsampled data
# due to the high computational complexity of SVMs.
from sklearn.svm import LinearSVC

X_svm_tr, y_svm_tr = stratified_subsample(X_train, y_train, n=300_000)

svm_linear = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0.0)),
    ("scaler", StandardScaler()),
    ("clf", LinearSVC(
        class_weight="balanced",
        random_state=42
    ))
])

svm_linear.fit(X_svm_tr, y_svm_tr)

X_rbf_tr, y_rbf_tr = stratified_subsample(X_train, y_train, n=30_000)

svm_rbf = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0.0)),
    ("scaler", StandardScaler()),
    ("clf", SVC(
        kernel="rbf",
        probability=False,
        class_weight="balanced",
        cache_size=2000,
        random_state=42
    ))
])

svm_rbf.fit(X_rbf_tr, y_rbf_tr)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('imputer', ...), ('scaler', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'constant'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",0.0
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. 

In [24]:
''' These cells train a Gradient Boosting classifier.
 Regularization and subsampling are used to control model complexity
and reduce overfitting.'''
# Check availability of boosting libraries
HAS_XGB = False
HAS_LGBM = False

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    pass

try:
    from lightgbm import LGBMClassifier
    HAS_LGBM = True
except ImportError:
    pass

print("XGBoost available:", HAS_XGB)
print("LightGBM available:", HAS_LGBM)


XGBoost available: True
LightGBM available: True


In [25]:
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
scale_pos_weight = (neg / max(pos, 1))

if HAS_XGB:
    gb = XGBClassifier(
        n_estimators=600,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        reg_alpha=0.0,
        scale_pos_weight=scale_pos_weight,
        eval_metric="logloss",
        random_state=42,
        n_jobs=-1
    )
elif HAS_LGBM:
    gb = LGBMClassifier(
        n_estimators=1200,
        learning_rate=0.03,
        num_leaves=63,
        subsample=0.8,
        colsample_bytree=0.8,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )
else:
    raise ImportError("Neither xgboost nor lightgbm is installed. Install one for the required boosting model.")

gb.fit(X_train, y_train)
type(gb)

xgboost.sklearn.XGBClassifier

In [26]:
def make_val_subsample(X_val, y_val, n=1_000_000, random_state=42):
    n = min(n, len(X_val))
    X_small, _, y_small, _ = train_test_split(
        X_val, y_val, train_size=n, stratify=y_val, random_state=random_state
    )
    return X_small, y_small

def evaluate_models_auto(models_dict, X_val, y_val, slow_models=None,
                         val_subsample_n=500_000, random_state=42,
                         show_plots=False):
    if slow_models is None:
        slow_models = set()

    need_sub = any(name in slow_models for name in models_dict.keys())
    X_val_small, y_val_small = (None, None)
    if need_sub:
        X_val_small, y_val_small = make_val_subsample(
            X_val, y_val, n=val_subsample_n, random_state=random_state
        )
        print(f"Validation subsample for slow models: {X_val_small.shape}")

    results = []
    for name, model in models_dict.items():
        if name in slow_models:
            X_eval, y_eval = X_val_small, y_val_small
            eval_note = f"val_subsample(n={len(X_eval)})"
        else:
            X_eval, y_eval = X_val, y_val
            eval_note = f"val_full(n={len(X_eval)})"

        print(f"Evaluating: {name} [{eval_note}]")
        r = evaluate_model(model, X_eval, y_eval, name, show_plots=show_plots)
        r["Eval_Set"] = eval_note
        results.append(r)

    return pd.DataFrame(results).sort_values("AP", ascending=False).reset_index(drop=True)


In [27]:
models = {
    "LogReg (L2, balanced)": logreg_l2,
    "KNN (scaled)": knn,
    "SVM (linear) [subsampled train]": svm_linear,
    "SVM (RBF) [subsampled train]": svm_rbf,
    "Decision Tree": dt,
    "Random Forest (subsampled train)": rf,
}

# Add gb if defined
if "gb" in globals():
    models["Gradient Boosting (XGB/LGBM)"] = gb

slow_models = {
    "KNN (scaled)",
    "SVM (linear) [subsampled train]",
    "SVM (RBF) [subsampled train]",
    "Random Forest (subsampled train)"
}

results_df = evaluate_models_auto(
    models_dict=models,
    X_val=X_val,
    y_val=y_val,
    slow_models=slow_models,
    val_subsample_n=500_000,
    random_state=42,
    show_plots=False   # أسرع: لا ترسم لكل نموذج
)
results_df


Validation subsample for slow models: (500000, 8)
Evaluating: LogReg (L2, balanced) [val_full(n=6486898)]
Evaluating: KNN (scaled) [val_subsample(n=500000)]
Evaluating: SVM (linear) [subsampled train] [val_subsample(n=500000)]
Evaluating: SVM (RBF) [subsampled train] [val_subsample(n=500000)]
Evaluating: Decision Tree [val_full(n=6486898)]


MemoryError: Unable to allocate 396. MiB for an array with shape (8, 6486898) and data type float64

In [ ]:
from sklearn.model_selection import train_test_split

VAL_SUB_N = 500_000

X_val_small, _, y_val_small, _ = train_test_split(
    X_val, y_val,
    train_size=min(VAL_SUB_N, len(X_val)),
    stratify=y_val,
    random_state=42
)

print("X_val_small:", X_val_small.shape)


In [ ]:
results = []

# Fast-ish on full validation
results.append(evaluate_model(logreg_l2, X_val, y_val, "LogReg (L2, balanced)", show_plots=False))
results.append(evaluate_model(dt,       X_val, y_val, "Decision Tree", show_plots=False))

# Slow models on subsample
results.append(evaluate_model(knn,       X_val_small, y_val_small, "KNN (val_subsample)", show_plots=False))
results.append(evaluate_model(svm_linear,X_val_small, y_val_small, "SVM linear (val_subsample)", show_plots=False))
results.append(evaluate_model(svm_rbf,   X_val_small, y_val_small, "SVM RBF (val_subsample)", show_plots=False))
results.append(evaluate_model(rf,        X_val_small, y_val_small, "Random Forest (val_subsample)", show_plots=False))

results_df = pd.DataFrame(results).sort_values("AP", ascending=False).reset_index(drop=True)
results_df


In [ ]:
best_name = results_df.iloc[0]["Model"]
print("Best model:", best_name)

if "subsample" in best_name:
    _ = evaluate_model(models[best_name] if "models" in globals() else None,
                       X_val_small, y_val_small, best_name, show_plots=True)
else:
    _ = evaluate_model(logreg_l2 if "LogReg" in best_name else dt,
                       X_val, y_val, best_name, show_plots=True)


In [ ]:
def best_threshold_by_f1(y_true, y_prob):
    thresholds = np.linspace(0.05, 0.95, 91)
    best = (None, -1)
    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        f1 = f1_score(y_true, y_pred, zero_division=0)
        if f1 > best[1]:
            best = (t, f1)
    return best

best_model_name = results_df.iloc[0]["Model"]
best_model = {
    "LogReg (L2, balanced)": logreg_l2,
    "KNN (scaled)": knn,
    "SVM (linear) [subsampled train]": svm_linear,
    "SVM (RBF) [subsampled train]": svm_rbf,
    "Decision Tree": dt,
    "Random Forest": rf,
    "Gradient Boosting (XGB/LGBM)": gb
}[best_model_name]

_, best_prob = get_pred_and_proba(best_model, X_val)
t, f1 = best_threshold_by_f1(y_val, best_prob)
print("Best model:", best_model_name)
print("Best threshold by F1:", t, "F1:", f1)